In [1]:
import requests
import pandas as pd
import sqlite3
import lxml.html

In [2]:
r = requests.get("https://projects.fivethirtyeight.com/global-club-soccer-rankings/")

In [3]:
html = lxml.html.fromstring(r.text)
rank_table = html.get_element_by_id("all-teams-table")
table_head = rank_table.getchildren()[0].getchildren()
header = rank_table.getchildren()[0].getchildren()[1]
table_body = rank_table.getchildren()[1].getchildren()

In [4]:
data = {
    "rank": [],
    "change": [],
    "team": [],
    "league": [],
    "country": [],
    "spi": []
}

In [5]:
for row in table_body:
    data["rank"].append(row[0].text_content().strip())
    data["change"].append(row[1].text_content().strip())
    data["team"].append(row[2].text_content().strip())
    data["league"].append(row[3].text_content().strip())
    data["country"].append(row[4].text_content().strip())
    data["spi"].append(row[7].text_content().strip())

In [6]:
df = pd.DataFrame(data)

In [7]:
df.league.value_counts()

Super League             40
Superliga                38
Premier League           36
USL Championship         36
Bundesliga               30
Championship             24
League Two               24
Major League Soccer      24
League One               24
La Liga 2                22
Serie A                  20
La Liga                  20
Serie B                  20
Ligue 1                  20
Brasileirão              20
Liga MX                  20
Ligue 2                  20
Süper Lig                19
2. Bundesliga            18
J1 League                18
Eredivisie               18
Primeira Liga            18
Premier Division         16
Eliteserien              16
Allsvenskan              16
UEFA Europa League       16
First Division A         16
Premiership              12
A-League                 10
UEFA Champions League     1
Name: league, dtype: int64

In [8]:
english_league = df[(df.league=="Premier League") & (df.country=="England")]

for row in english_league.itertuples():
    print(row[1:])

('1', '', 'Man. City', 'Premier League', 'England', '94.2')
('3', '', 'Liverpool', 'Premier League', 'England', '92.4')
('7', '-1', 'Chelsea', 'Premier League', 'England', '85.9')
('9', '-1', 'Tottenham', 'Premier League', 'England', '85.1')
('17', '', 'Man. United', 'Premier League', 'England', '80.0')
('18', '', 'Arsenal', 'Premier League', 'England', '79.7')
('20', '-1', 'Everton', 'Premier League', 'England', '78.7')
('27', '-1', 'Leicester', 'Premier League', 'England', '75.3')
('30', '+1', 'Wolves', 'Premier League', 'England', '74.0')
('43', '+3', 'Crystal Palace', 'Premier League', 'England', '71.0')
('48', '+5', 'Newcastle', 'Premier League', 'England', '70.3')
('53', '+1', 'West Ham', 'Premier League', 'England', '70.0')
('54', '+1', 'Bournemouth', 'Premier League', 'England', '69.6')
('58', '+3', 'Southampton', 'Premier League', 'England', '68.6')
('62', '+3', 'Watford', 'Premier League', 'England', '67.6')
('63', '+4', 'Burnley', 'Premier League', 'England', '67.6')
('80', 

In [9]:
db = sqlite3.connect(":memory:")
cursor = db.cursor()
cursor.execute("""
    CREATE TABLE ENGLISH_LEAGUE(RANK, CHANGE, TEAM, LEAGUE, COUNTRY, SPI)
""")

for row in df[(df.league=="Premier League") & (df.country=="England")].itertuples():
    insert_sql_syntax = """
        INSERT INTO ENGLISH_LEAGUE(RANK, CHANGE, TEAM, LEAGUE, COUNTRY, SPI)
        VALUES(?,?,?,?,?,?)
    """
    cursor.execute(insert_sql_syntax, row[1:])
    
db.commit() 

In [10]:
for row in cursor.execute("""
    SELECT *
    FROM ENGLISH_LEAGUE
    WHERE
    SPI >= "80"
"""):
    print(row)

('1', '', 'Man. City', 'Premier League', 'England', '94.2')
('3', '', 'Liverpool', 'Premier League', 'England', '92.4')
('7', '-1', 'Chelsea', 'Premier League', 'England', '85.9')
('9', '-1', 'Tottenham', 'Premier League', 'England', '85.1')
('17', '', 'Man. United', 'Premier League', 'England', '80.0')


In [11]:
db.close()